In [ ]:
%pip install -U transformers

## from repo

In [ ]:
import os

os.makedirs('data')

In [ ]:
%%capture

!mkdir data/copa/
!cd data/copa/
!wget https://asgordon.github.io/downloads/COPA-resources.tgz
!tar -xvf COPA-resources.tgz
!cp COPA-resources/datasets/copa-{dev,test}.xml data/copa/
!rm COPA-resources.tgz
!rm -r COPA-resources/
# ===
!mkdir data/storycloze/
!wget https://github.com/snigdhac/StoryComprehension_EMNLP/raw/master/Dataset/RoCStories/cloze_test_val__spring2016%20-%20cloze_test_ALL_val.tsv -O data/storycloze/dev.tsv
!wget https://github.com/snigdhac/StoryComprehension_EMNLP/raw/master/Dataset/RoCStories/test_spring2016.tsv -O data/storycloze/test.tsv
# ===
!mkdir data/agn/
!wget https://github.com/tonyzhaozh/few-shot-learning/raw/main/data/agnews/train.csv -O data/agn/dev.csv
!wget https://github.com/tonyzhaozh/few-shot-learning/raw/main/data/agnews/test.csv -O data/agn/test.csv
# ===
!mkdir data/arc-easy/
!mkdir data/arc-challenge/
!wget https://ai2-public-datasets.s3.amazonaws.com/arc/ARC-V1-Feb2018.zip
!unzip ARC-V1-Feb2018.zip
!cp ARC-V1-Feb2018-2/ARC-Easy/ARC-Easy-Dev.jsonl data/arc-easy/dev.jsonl
!cp ARC-V1-Feb2018-2/ARC-Easy/ARC-Easy-Test.jsonl data/arc-easy/test.jsonl
!cp ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Dev.jsonl data/arc-challenge/dev.jsonl
!cp ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Test.jsonl data/arc-challenge/test.jsonl
!rm ARC-V1-Feb2018.zip
!rm -r __MACOSX/ ARC-V1-Feb2018-2/
# ===
!mkdir data/boolq
!wget https://storage.cloud.google.com/boolq/dev.jsonl
!cp dev.jsonl data/boolq/dev.jsonl
!rm dev.jsonl
# ===
!mkdir data/cb
!wget https://dl.fbaipublicfiles.com/glue/superglue/data/v2/CB.zip
!unzip CB.zip
!cp CB/val.jsonl data/cb/dev.jsonl
!rm CB.zip
!rm -r CB/
# ===
!mkdir data/cqa/
!wget https://s3.amazonaws.com/commensenseqa/dev_rand_split.jsonl -O data/cqa/dev.jsonl
# ===
!mkdir data/hellaswag/
!wget https://github.com/rowanz/hellaswag/raw/master/data/hellaswag_val.jsonl -O data/hellaswag/dev.jsonl
# ===
!mkdir data/obqa
!wget https://ai2-public-datasets.s3.amazonaws.com/open-book-qa/OpenBookQA-V1-Sep2018.zip
!unzip OpenBookQA-V1-Sep2018.zip
!cp OpenBookQA-V1-Sep2018/Data/Main/dev.jsonl data/obqa/dev.jsonl
!cp OpenBookQA-V1-Sep2018/Data/Main/test.jsonl data/obqa/test.jsonl
!rm OpenBookQA-V1-Sep2018.zip
!rm -r OpenBookQA-V1-Sep2018/
# ===
!mkdir data/race-m/
!mkdir data/race-h/
!wget http://www.cs.cmu.edu/~glai1/data/race/RACE.tar.gz
!tar -xvf RACE.tar.gz
!cp -r RACE/{dev,test} data/race-m/
!cp -r RACE/{dev,test} data/race-h/
!rm RACE.tar.gz
!rm -r RACE/
# ===
!mkdir data/rte
!wget https://dl.fbaipublicfiles.com/glue/superglue/data/v2/RTE.zip
!unzip RTE.zip
!cp RTE/val.jsonl data/rte/dev.jsonl
!rm -r RTE/
!rm RTE.zip
# ===
!mkdir data/sst-2/
!mkdir data/sst-5/
!wget https://raw.githubusercontent.com/prrao87/fine-grained-sentiment/master/data/sst/sst_dev.txt -O data/sst-2/dev.tsv
!wget https://raw.githubusercontent.com/prrao87/fine-grained-sentiment/master/data/sst/sst_test.txt -O data/sst-2/test.tsv
!wget https://raw.githubusercontent.com/prrao87/fine-grained-sentiment/master/data/sst/sst_train.txt -O data/sst-2/train.tsv
!cp data/sst-2/{dev,test,train}.tsv data/sst-5/
# ===
!mkdir data/trec/
!wget https://raw.githubusercontent.com/tonyzhaozh/few-shot-learning/main/data/trec/train.txt -O data/trec/dev.txt
!wget https://raw.githubusercontent.com/tonyzhaozh/few-shot-learning/main/data/trec/test.txt -O data/trec/test.txt

In general, examples should be of the form:

        {
        'options': [opt_1, opt_2, ..., opt_m]
        'label': l  # index of correct option
        }

opt_i is an option of the form:

        {
        'premise': premise # the question premise (string)
        'hypothesis': h # hypothesis answer (str) we calculate conditional P(hypothesis|premise)
        'unc_presmise': up # the premise for calculating uncond likelihood (str)
        'unc_hypothesis': uh # the hypothesis used for calculating uncond likelihood P(hypothesis)
                             # this will often just be hypothesis but may differ slightly for format
        
        }

### score.py

In [ ]:
#from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    LlamaForCausalLM,
    LlamaTokenizer,
    GPT2LMHeadModel,
    GPT2Tokenizer
)
from utils import score
import numpy as np
import torch

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def get_model(model_name):
    if model_name.lower() in ['flan-t5-small', 'small', 't5-small']:
        # Flan-T5-Small
        model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
        encoder = AutoTokenizer.from_pretrained("google/flan-t5-small")
        model.to(DEVICE)
        model.eval()
        name    = 'Flan-T5-Small'
    elif model_name.lower() in ['flan-t5-large', 'large', 't5-large']:
        # Flan-T5-Small
        model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
        encoder = AutoTokenizer.from_pretrained("google/flan-t5-large")
        model.to(DEVICE)
        model.eval()
        name    = 'Flan-T5-Large'
    elif model_name.lower() in ['t0','mt0','mt0-large']:
        # T0
        model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/mt0-large",
                                                      torch_dtype="auto",
                                                      device_map="auto")
        encoder = AutoTokenizer.from_pretrained("bigscience/mt0-large")
        model.to(DEVICE)
        model.eval()
        name    = 'mT0-large'
    elif(model_name.lower() in ['llama', 'llama3', 'llama3-1b', 'llama-3', 'llama-3-1b']):
        model = LlamaForCausalLM.from_pretrained(
                "meta-llama/Llama-3.2-1B",
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                device_map="auto" if torch.cuda.is_available() else None)
        #encoder = LlamaTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
        encoder = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
        name    = 'Llama-3-1B'
    elif model_name.lower() in ['gpt2', 'gpt2-s', 'gpt2-small', 'gs', 's', 'small']:
        # GPT-2 Small
        model   = GPT2LMHeadModel.from_pretrained('gpt2').cuda(0).eval()
        encoder = GPT2Tokenizer.from_pretrained('gpt2')
        name    = 'GPT-2-Small'
    return model, encoder, name

class ArgsSubst():
    def __init__(self, dataset:str, model:str, n_shot:int=0, variant:str=None,
                 splt:str='dev', batch:int=10, sample:int=None, seed:int=0,
                 key:str=''):

        self.dataset=dataset
        self.model=model
        self.n_shot=n_shot
        self.variant=variant
        self.splt=splt
        self.batch=batch
        self.sample=sample
        self.seed=seed
        self.key=key

In [ ]:
from data_loaders import *

def get_examples(dataset_name, split, stem, n_shot, variant):
    if dataset_name == 'copa':
        examples = load_examples_copa(f'{stem}copa-{split}.xml')
        closed_label_space = False
    elif dataset_name == 'copa-rev':
        examples = load_examples_copa_rev(f'{stem}copa-{split}.xml')
        closed_label_space = False
    elif dataset_name == 'storycloze':
        examples = load_examples_storycloze(f'{stem}{split}.tsv')
        closed_label_space = False
    elif dataset_name == 'hellaswag':
        examples = load_examples_hellaswag(f'{stem}dev.jsonl')
        closed_label_space = False
    elif dataset_name == 'race-m' or \
         dataset_name == 'race-h':
        version = 'high' if dataset_name == 'race-h' else 'middle'
        examples = load_examples_race(stem, split, version)
        closed_label_space = False
    elif dataset_name == 'arc-easy' or \
        dataset_name == 'arc-challenge':
        examples = load_examples_arc(f'{stem}{split}.jsonl')
        closed_label_space = False
    elif dataset_name == 'obqa':
        examples = load_examples_obqa(f'{stem}{split}.jsonl')
        closed_label_space = False
    elif dataset_name == 'cqa':
        if args.split == 'test':
            raise NotImplementedError("CSQA does not release test answers directly, please do not spam their leaderboard either :)")
        else:
            examples = load_examples_cqa(f'{stem}{split}.jsonl')
        closed_label_space = False
    elif dataset_name == 'boolq':
        examples = load_examples_boolq(f'{stem}dev.jsonl')
        closed_label_space = True
    elif dataset_name == 'rte':
        examples = load_examples_rte(f'{stem}dev.jsonl')
        closed_label_space = True
    elif dataset_name == 'cb':
        examples = load_examples_cb(f'{stem}dev.jsonl')
        closed_label_space = True
    elif dataset_name == 'sst-2':
        if n_shot > 0:
            examples = load_examples_sst2(f'{stem}{split}.tsv', f'{stem}/train.tsv', n_shot)
        elif variant is not None:
            examples = load_examples_sst2_variants(f'{stem}{split}.tsv', variant)
        else:
            examples = load_examples_sst2(f'{stem}{split}.tsv')
        closed_label_space = True
    elif dataset_name == 'sst-5':
        examples = load_examples_sst5(f'{stem}{split}.tsv')
        closed_label_space = True
    elif dataset_name == 'agn':
        examples = load_examples_agn(f'{stem}{split}.csv')
        closed_label_space = True
    elif dataset_name == 'trec':
        split = 'train' if split == 'dev' else split
        examples = load_examples_trec(f'{stem}{split}.txt')
        closed_label_space = True
    else:
        raise ValueError(f'Unknown dataset {dataset_name}')

    return examples, closed_label_space

### run code

In [ ]:
import pandas as pd

experiments = pd.DataFrame(columns=['model','dataset','n_shot','UNC','LM','AVG','PMIdc','PMI'])

In [ ]:
# 'google/flan-t5-small' (não funciona)
# "meta-llama/Meta-Llama-3-1B"
# 'gpt2'
args = ArgsSubst(dataset='race-m', model='t0',seed=42, splt='test')#, sample=10)

#if args.debug:
#    pdb.set_trace()

os.environ['PYTHONHASHSEED'] = str(args.seed)
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)

model, encoder, name = get_model(args.model, args.key) # ok✅

In [ ]:
if args.dataset.endswith('-rev'):
    stem = f'data/{args.dataset[:-4]}/'
else:
    stem = f'data/{args.dataset}/'
examples, closed_label_space = get_examples(args.dataset, args.splt, stem, args.n_shot, args.variant) # ok✅ (acho)

In [ ]:
if args.sample:
    assert(args.sample <= len(examples))
    examples = random.sample(examples, args.sample)
accs = score(model, args.model, encoder, examples, stem, args.splt, args.batch) # TODO

# save results
if(args.sample is None) and (args.splt != 'dev'):
    experiments.loc[experiments.shape[0]+1,:] = [name,
                                                 args.dataset,
                                                 args.n_shot,
                                                 accs['domain_cond'],
                                                 accs['lm'],
                                                 accs['tok_mean'],
                                                 accs['dcpmi'],
                                                 accs['pmi'],
                                                 ]

# print results
print(f'{name} gets {accs}% on {args.dataset}')
print(f"{accs['domain_cond']} & {accs['lm']} & {accs['tok_mean']} & {accs['pmi']} & {accs['dcpmi']}")
print(f"{accs['domain_cond']}, {accs['lm']}, {accs['tok_mean']}, {accs['dcpmi']}")

MAKE SURE TOKENIZATION AND FORMATTING LOOKS OK

print example 0 of 1436:
CONDITIONAL:
For many Chinese, winter holiday season doesn't seem complete without the CCTV Spring Festival Gala. But it's impossible to satisfy all 1.3 billion Chinese. Even if you, especially young audience complain about the Spring Festival Gala, watching it while having Lunar New Year's Eve dinner has become a common way for families to get together. It's much more than just a TV program, it's also a way to bring family members together and get them talking during the holiday. Think about how you'll watch the gala and discover its fun. 1. Share your opinions about the programs and artists with your family members. Introduce artists who young people enjoy to the elderly memebers of your family, such as "fellow brother" William Chan, or former singing group EXO members Kris Wu and Lu Han. 2. Post your _ and complaints on micro blogs and WeChat moments to share with friends. Then they will know what you're satisf

100%|██████████| 1436/1436 [00:06<00:00, 234.75it/s]


actually calculating


100%|██████████| 1436/1436 [23:32<00:00,  1.02it/s] 


saving cache to data/race-m/t0-test.hist
mT0-large gets {'lm': 0.4596100278551532, 'tok_mean': 0.49025069637883006, 'dcpmi': 0.49303621169916434, 'pmi': 0.48746518105849584, 'domain_cond': 0.21518105849582173}% on race-m
0.21518105849582173 & 0.4596100278551532 & 0.49025069637883006 & 0.48746518105849584 & 0.49303621169916434
0.21518105849582173, 0.4596100278551532, 0.49025069637883006, 0.49303621169916434


### **Testes em massa**

In [ ]:
import pandas as pd

experiments = pd.DataFrame(columns=['model','dataset','n_shot','UNC','LM','AVG','PMIdc','PMI'])

In [ ]:
args = ArgsSubst(dataset='copa', model='t0',seed=42, splt='test')#, sample=10)

#if args.debug:
#    pdb.set_trace()

os.environ['PYTHONHASHSEED'] = str(args.seed)
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)

model, encoder, name = get_model(args.model, args.key) # ok✅

for dataset_name in ["copa", "copa-rev", "storycloze", "hellaswag"]:#, "race-m", "race-h", "arc-easy", "arc-challenge", "obqa", "cqa", "boolq", "rte", "cb", "sst-2", "sst-5", "agn", "trec"]:
    if dataset_name.endswith('-rev'):
        stem = f'data/{dataset_name[:-4]}/'
    else:
        stem = f'data/{dataset_name}/'
    examples, closed_label_space = get_examples(dataset_name, args.splt, stem, args.n_shot, args.variant)

    accs = score(model, args.model, encoder, examples, stem, args.splt, args.batch)

    # save results
    if(args.sample is None) and (args.splt != 'dev'):
        experiments.loc[experiments.shape[0]+1,:] = [name,
                                                    dataset_name,
                                                    args.n_shot,
                                                    accs['domain_cond'],
                                                    accs['lm'],
                                                    accs['tok_mean'],
                                                    accs['dcpmi'],
                                                    accs['pmi'],
                                                    ]


MAKE SURE TOKENIZATION AND FORMATTING LOOKS OK

print example 0 of 500:
CONDITIONAL:
The item was packaged in bubble wrap because</s><BREAK>it was fragile.</s>
UNCONDITIONAL:
because</s><BREAK>it was fragile.</s>
logging examples


100%|██████████| 500/500 [00:00<00:00, 2048.04it/s]


actually calculating


100%|██████████| 500/500 [00:41<00:00, 12.19it/s]


saving cache to data/copa/t0-test.hist
Loading existing cache, size 0.707429 MB
MAKE SURE TOKENIZATION AND FORMATTING LOOKS OK

print example 0 of 500:
CONDITIONAL:
it was fragile so</s><BREAK>the item was packaged in bubble wrap.</s>
UNCONDITIONAL:
so</s><BREAK>the item was packaged in bubble wrap.</s>
logging examples


...

saving cache to data/storycloze/t0-test.hist
MAKE SURE TOKENIZATION AND FORMATTING LOOKS OK

print example 0 of 10042:
CONDITIONAL:
A man is sitting on a roof. he</s><BREAK>is using wrap to wrap a pair of skis.</s>
UNCONDITIONAL:
he</s><BREAK>is using wrap to wrap a pair of skis.</s>
logging examples


100%|██████████| 10042/10042 [00:27<00:00, 359.21it/s]


actually calculating


Streaming output truncated to the last 5000 lines.
100%|██████████| 10042/10042 [1:22:59<00:00,  2.02it/s]


saving cache to data/hellaswag/t0-test.hist


## Results:

In [ ]:
experiments.to_csv('experiments_results.csv')

GPT-2:

        saving cache to data/copa/gpt2-dev.hist
        ==================================================
        G-S gets {'lm': 0.61, 'tok_mean': 0.632, 'dcpmi': 0.628, 'pmi': 0.64, 'domain_cond': 0.564}% on copa
        0.564 & 0.61 & 0.632 & 0.64 & 0.628
        0.564, 0.61, 0.632, 0.628

---

### Llama-3:

- COPA

        saving cache to data/copa/llama3-dev.hist
        ==================================================
        Llama-3 gets {'lm': 0.782, 'tok_mean': 0.67, 'dcpmi': 0.806, 'pmi': 0.802, 'domain_cond': 0.578}% on copa
        0.578 & 0.782 & 0.67 & 0.802 & 0.806
        0.578, 0.782, 0.67, 0.806

        saving cache to data/copa/llama3-test.hist
        ==================================================
        Llama-3 gets {'lm': 0.836, 'tok_mean': 0.692, 'dcpmi': 0.846, 'pmi': 0.854, 'domain_cond': 0.598}% on copa
        0.598 & 0.836 & 0.692 & 0.854 & 0.846
        0.598, 0.836, 0.692, 0.846

- COPA (reversed)

        saving cache to data/copa/llama3-dev.hist
        ==================================================
        Llama-3 gets {'lm': 0.772, 'tok_mean': 0.772, 'dcpmi': 0.772, 'pmi': 0.772, 'domain_cond': 0.5}% on copa-rev
        0.5 & 0.772 & 0.772 & 0.772 & 0.772
        0.5, 0.772, 0.772, 0.772

        saving cache to data/copa/llama3-test.hist
        ==================================================
        Llama-3 gets {'lm': 0.806, 'tok_mean': 0.806, 'dcpmi': 0.806, 'pmi': 0.806, 'domain_cond': 0.5}% on copa-rev
        0.5 & 0.806 & 0.806 & 0.806 & 0.806
        0.5, 0.806, 0.806, 0.806

---